In [1]:
#@title Download data

!git clone https://github.com/dataconHack/hackathon.git

Cloning into 'hackathon'...
remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 15 (delta 3), reused 2 (delta 0), pack-reused 0
Unpacking objects: 100% (15/15), 22.91 KiB | 2.86 MiB/s, done.


In [2]:
#@title Installs
%%capture

!pip install klib

In [3]:
#@title Imports

import pandas as pd
import klib

In [4]:
# основной датасет (экспериментальные данные)
data = pd.read_csv('/content/hackathon/data.csv')
data.drop(['Unnamed: 0.1', 'Unnamed: 0'], axis=1, inplace=True)

# удалим дубликаты
data.drop_duplicates(inplace=True)

# дропнем сэмплы, где отсутствуют значения для таргетной переменной
data.dropna(subset=['ZOI_drug_NP'], inplace=True)

# преобразуем тип данных для таргетной переменной
to_replace = {'32+' : '32',
              '17+2' : '19'}

data.replace(to_replace=to_replace, inplace=True)
data['ZOI_drug_NP'] = data['ZOI_drug_NP'].astype(float)

# дропнем фичи с большим количеством пропусков
data.drop(['ZOI_NP'], axis=1, inplace=True)
data.drop(['fold_increase_in_antibacterial_activity (%)'], axis=1, inplace=True)

# предполагая важность следующих фичей, дропним сэмплы, в которых они отсутствуют
# в дальнейшем при анализе необходимо будет оценить их важность и либо оставить всё как есть,
# либо вернуть удаленные сэмплы и дропнуть сами фичи
data = data[(data['NP_concentration'].notnull()) & (data['Drug_dose'].notnull() ) & (data['ZOI_drug']).notnull()]
klib.missingval_plot(data, figsize=(15,10), sort=True)

Displaying only columns with missing values.
No missing values found in the dataset.


In [6]:
# датасет с информацией по лекарствам
drug_descriptors = pd.read_csv('/content/hackathon/drug_descriptors.csv')
drug_descriptors.drop(['Unnamed: 0'], axis=1, inplace=True)
drug_descriptors.rename(columns={'drug' : 'Drug'}, inplace=True)
klib.missingval_plot(drug_descriptors)

No missing values found in the dataset.


In [7]:
# датасет с информацией о бактериях
bacterial_descriptors = pd.read_csv('/content/hackathon/bacterial_descriptors.csv')

# дропнем фичи с большим количеством пропусков
bacterial_descriptors.drop(['subkingdom', 'clade'], axis=1, inplace=True)

# так как в датасете содержатся не только бактерии, но и грибы, для которых параметр gram не определен
# заменим отсутствующие значения на новую категорию - undefined
bacterial_descriptors['gram'] = bacterial_descriptors['gram'].fillna('undefined')
klib.missingval_plot(drug_descriptors)

No missing values found in the dataset.


In [8]:
# исправим несоответсвия в названиях бактерий для корректого мерджа
to_replace = {'Acinetobacter baumanii' : 'Acinetobacter baumannii',
              'Bacillus spp.        ' : 'Bacillus sp.',
              'Enterobacter cloacae ' : 'Enterobacter cloacae',
              'Salmonella  typhi' : 'Salmonella typhi'}

data.replace(to_replace=to_replace, inplace=True)

In [9]:
# присоединим датасет по бактериям к основному датасету
data2 = pd.merge(data, bacterial_descriptors, how='left', on=['Bacteria'])

In [11]:
# присоединим датасет по лекарствам к ранее объединенному датасету
data3 = pd.merge(data2, drug_descriptors, how='left', on=['Drug'])